In [1]:
from keras import backend as K
import tensorflow as tf

def weighted_binary_crossentropy(y_true, y_pred):
    x = 0.10 #Approximate percentage of positives in each of the 17 zones
    POS_ADJ = 0.5/x
    NEG_ADJ = 0.5/(1-x)
    n_values = y_true.shape[0]
    elems = (tf.unstack(y_true,axis=0)) 
    adj = tf.map_fn(lambda x:tuple([tf.cond(tf.equal(x[i],1.),lambda:POS_ADJ,lambda: NEG_ADJ) for i in range(n_values)]),
                    elems, 
                    dtype = tuple([tf.float32 for i in range(n_values)]) )
    adj = tf.stack(adj,axis=0)
    print("Before averaging")
    print(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)).eval())
    return K.mean(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)),axis=-1)

Using TensorFlow backend.


In [3]:
import numpy as np
sess = tf.InteractiveSession()
a= np.array([[1,0,1,1],[1,1,1,1],[0,0,0,0]])
print(a.shape)
b= np.array([[0.5,0.5,0.5,0.5],[0.01,0.1,0.3,0.9],[0.5,0.7,0.5,0.2]])

print(b.shape)
z = weighted_binary_crossentropy(tf.convert_to_tensor(a,dtype=tf.float32),tf.convert_to_tensor(b,dtype=tf.float32))
print("Final result")
print(z.eval())

(3, 4)
(3, 4)
Before averaging
[[  3.46573591   0.3850818    3.46573591   3.46573591]
 [ 23.0258522   11.5129261    6.01986408   0.52680266]
 [  0.3850818    0.66887385   0.3850818    0.12396865]]
Final result
[  2.69557238  10.27136135   0.39075151]
